In [1]:
from pyspark.sql import SparkSession

In [2]:
features = ['other_pp','from_fk','from_ti','from_corner','from_counter','from_gk','from_keeper','from_ko',
            'header','corner_type','fk_type','pk_type',
            'half_volley_technique','volley_technique','lob_technique','overhead_technique','backheel_technique','diving_h_technique',
            'distance_to_goal', 'shot_angle', 'preferred_foot_shot', 'under_pressure',
            'shot_aerial_won','shot_first_time','shot_one_on_one','shot_open_goal','shot_follows_dribble','players_inside_area']
target = ['goal']

In [3]:
spark = SparkSession.builder.appName("xG6").getOrCreate()
spark.conf.set("spark.sql.debug.maxToStringFields", 1000)

your 131072x1 screen size is bogus. expect trouble
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/26 02:17:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
events = spark.read.csv('Datas/events_shot.csv', header=True, inferSchema=True)

In [5]:
import pre_training as pt

In [6]:
train_data, test_data = pt.pre_training(events, features)

In [7]:
trained_model = pt.train_model(train_data,'gbt')

In [8]:
predictions = trained_model.transform(test_data)

In [21]:
from pyspark.sql.functions import udf, col, format_number
from pyspark.sql.types import DoubleType

# Define the function to extract the second element from the probability list
def extract_goal_probability(probability):
    return float(probability[1])

# Register the function as a UDF
extract_goal_probability_udf = udf(extract_goal_probability, DoubleType())

# Overwrite the prediction column using the UDF
predictions = predictions.withColumn("goal_probability", extract_goal_probability_udf(col("probability")))

# Format the goal_probability to remove scientific notation
predictions = predictions.withColumn("goal_probability", format_number(col("goal_probability"), 10))
# convert goal_probability to float
predictions = predictions.withColumn("goal_probability", col("goal_probability").cast(DoubleType()))

In [22]:
from model_evaluation import ModelEvaluation

In [23]:
me = ModelEvaluation(predictions,'goal','prediction')

In [24]:
me_r = ModelEvaluation(predictions,'shot_statsbomb_xg','goal_probability',model_type='regression')

In [25]:
me_r.get_all_metrics()

{'MSE': 0.0040395032176577285,
 'RMSE': 0.06355708628986802,
 'MAE': 0.04298674341053067,
 'R2': 0.8129201469995636}

In [26]:
me.get_all_metrics()

{'Accuracy': 0.9018510900863842,
 'Precision': 0.7774086378737541,
 'Recall': 0.17205882352941176,
 'Sensitivity': 0.17205882352941176,
 'Specificity': 0.9937934228809634,
 'F1': 0.28175797712221556,
 'FPR': 0.006206577119036591,
 'FNR': 0.8279411764705882}